In [10]:
from pyspark.sql import SparkSession
import json,time,os,subprocess

if __name__ == "__main__":
    # file_path_sec = "/opt/spark/data/xcom_data.json"
    file_path_sec = "/home/aky/data/xcom_data.json"
    if os.path.exists(file_path_sec):
        print("################# 존재합니다. spark 시작합니다 #################")
        spark = SparkSession.builder.appName("DataProcessing") \
                .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
                .getOrCreate()
                        
        with open(file_path_sec, 'r') as f:
            json_data = json.load(f)
            data = []
            for i in json_data:
                i_replace = i.replace("'",'"')
                data.append(json.loads(i_replace))
            df = spark.createDataFrame(data)
            json_list = [row.asDict() for row in df.collect()]
            print(type(json_list))
    else:
        print("################# 존재하지 않습니다 #################")


################# 존재합니다. spark 시작합니다 #################
<class 'list'>
